# Final Model Prediction

In [6]:
import sys
sys.path.append('a/UltimeTradingBot/Crypto_backtest_tools')
import pandas as pd
import json
import numpy as np
import urllib
#import ccxt
import ccxt.async_support as ccxt  
import random
from keras.models import load_model
import os
import asyncio
import tensorflow as tf

## Garbage collection

In [7]:
import gc
gc.collect()    

42

## Local Functions first

In [8]:
# Get list of all IDs on binance
def give_first_kline_open_stamp(interval, symbol, start_ts=1499990400000):
        '''
        Returns the first kline from an interval and start timestamp and symbol
        :param interval:  1w, 1d, 1m etc - the bar length to query
        :param symbol:    BTCUSDT or LTCBTC etc
        :param start_ts:  Timestamp in miliseconds to start the query from
        :return:          The first open candle timestamp
        '''

        url_stub = "http://api.binance.com/api/v1/klines?interval="

        #/api/v1/klines?interval=1m&startTime=1536349500000&symbol=ETCBNB
        addInterval   = url_stub     + str(interval) + "&"
        addStarttime  = addInterval   + "startTime="  + str(start_ts) + "&"
        addSymbol     = addStarttime + "symbol="     + str(symbol)
        url_to_get = addSymbol

        kline_data = urllib.request.urlopen(url_to_get).read().decode("utf-8")
        kline_data = json.loads(kline_data)

        return kline_data[0][0]

def get_crypto_metadata(pair_list):
    Binance_USDT_HALAL=pair_list
    ids = []
    #ids = all_ids()
    for halalpair in Binance_USDT_HALAL:
    #    print( halalpair.replace('/',''))
        ids.append(halalpair.replace('/',''))
    #print(ids)
    MetaData=pd.DataFrame(ids)
    MetaData["Pair"]=Binance_USDT_HALAL
    counters=0
    for this_id in ids:
        '''
        Find launch Week of symbol, start at Binance launch date 2017-07-14 (1499990400000)
        Find launch Day of symbol in week
        Find launch minute of symbol in day
        '''

        symbol_launch_week_stamp   = give_first_kline_open_stamp('1w', this_id, 1499990400000 )
        symbol_launch_day_stamp    = give_first_kline_open_stamp('1d', this_id, symbol_launch_week_stamp)
        symbol_launch_minute_stamp = give_first_kline_open_stamp('1m', this_id, symbol_launch_day_stamp)
        MetaData.loc[counters,"launch_week_stamp"]=str(symbol_launch_week_stamp)
        MetaData.loc[counters,"launch_day_stamp"]=str(symbol_launch_day_stamp)
        MetaData.loc[counters,"launch_minute"]=pd.to_datetime(symbol_launch_minute_stamp, unit='ms')

        counters += 1

        #print("Week stamp", symbol_launch_week_stamp)
        #print("Day  stamp", symbol_launch_day_stamp)
        #print("Min  stamp", symbol_launch_minute_stamp)
        print(counters,end=" ")
        #print(this_id, "launched", symbol_launch_minute_stamp )
    return MetaData
    #print("")
    
def instant_pair_data(pair="GMT/BUSD",exchange=ccxt.binance(),window=15):
    ex=exchange
    ticker = ex.fetch_ticker(pair)
    pair_current_price=ticker['info']['askPrice']
    #print(pair_current_price)

    ohlcv1m = ex.fetch_ohlcv(pair, '1m', limit=15)
    ohlcv5m = ex.fetch_ohlcv(pair, '5m', limit=15)
    ohlcv15m = ex.fetch_ohlcv(pair, '15m', limit=15)
    ohlcv1h = ex.fetch_ohlcv(pair, '1h', limit=15)
    ohlcv1d = ex.fetch_ohlcv(pair, '1d', limit=15)

    pair_data=pd.DataFrame()
    pair_data.loc[0,"price"]=float(pair_current_price)
    #minute
    for window_i in range(1,window+1):
        pair_data.loc[0,"high-"+str(window_i)]=ohlcv1m[-window_i][1]
        pair_data.loc[0,"low-"+str(window_i)]=ohlcv1m[-window_i][2]
        pair_data.loc[0,"open-"+str(window_i)]=ohlcv1m[-window_i][0]
        pair_data.loc[0,"close-"+str(window_i)]=ohlcv1m[-window_i][3]
        if(window_i!=1):pair_data.loc[0,"volume-"+str(window_i)]=ohlcv1m[-window_i][4]

    for window_i in range(1,window+1):
        pair_data.loc[0,"high-"+str(window_i)+"_day"]=ohlcv1d[-window_i][1]
        pair_data.loc[0,"low-"+str(window_i)+"_day"]=ohlcv1d[-window_i][2]
        pair_data.loc[0,"open-"+str(window_i)+"_day"]=ohlcv1d[-window_i][0]
        pair_data.loc[0,"close-"+str(window_i)+"_day"]=ohlcv1d[-window_i][3]
        if(window_i!=1):pair_data.loc[0,"volume-"+str(window_i)+"_day"]=ohlcv1d[-window_i][4]  

    for window_i in range(1,window+1):
        pair_data.loc[0,"high-"+str(window_i)+"_hour"]=ohlcv1h[-window_i][1]
        pair_data.loc[0,"low-"+str(window_i)+"_hour"]=ohlcv1h[-window_i][2]
        pair_data.loc[0,"open-"+str(window_i)+"_hour"]=ohlcv1h[-window_i][0]
        pair_data.loc[0,"close-"+str(window_i)+"_hour"]=ohlcv1h[-window_i][3]
        if(window_i!=1):pair_data.loc[0,"volume-"+str(window_i)+"_hour"]=ohlcv1h[-window_i][4]  

    for window_i in range(1,window+1):
        pair_data.loc[0,"high-"+str(window_i)+"_15min"]=ohlcv15m[-window_i][1]
        pair_data.loc[0,"low-"+str(window_i)+"_15min"]=ohlcv15m[-window_i][2]
        pair_data.loc[0,"open-"+str(window_i)+"_15min"]=ohlcv15m[-window_i][0]
        pair_data.loc[0,"close-"+str(window_i)+"_15min"]=ohlcv15m[-window_i][3]
        if(window_i!=1):pair_data.loc[0,"volume-"+str(window_i)+"_15min"]=ohlcv5m[-window_i][4]  

    for window_i in range(1,window+1):
        pair_data.loc[0,"high-"+str(window_i)+"_5min"]=ohlcv5m[-window_i][1]
        pair_data.loc[0,"low-"+str(window_i)+"_5min"]=ohlcv5m[-window_i][2]
        pair_data.loc[0,"open-"+str(window_i)+"_5min"]=ohlcv5m[-window_i][0]
        pair_data.loc[0,"close-"+str(window_i)+"_5min"]=ohlcv5m[-window_i][3]
        if(window_i!=1):pair_data.loc[0,"volume-"+str(window_i)+"_5min"]=ohlcv5m[-window_i][4]   
    return  pair_data

def normalize(dataset,file='Normalization_Values.json'):
    global Normalization
    try:
        N=Normalization
    except:
        Normalization=None
    if(Normalization==None):
        print('Loading normalization from file')
        with open(file) as json_file:
            Normalization = json.load(json_file)
    else:
        print('normalization is loaded')

    mean=np.array(Normalization["mean"])
    std=np.array(Normalization["std"])
    dataset -= mean 
    dataset /= std
    return(dataset)

def Buy_Dessision(input):
    A=np.array(input)
    A = A.reshape(1,A.shape[0])
    predictions = model.predict(normalize(A))
    rounded = [round(x[0]) for x in predictions]
    return(rounded[0])

def Buy_Dessision_Normalized(input):
    A=np.array(input)
    A = A.reshape(1,A.shape[0])
    predictions = model.predict(A)
    rounded = [round(x[0]) for x in predictions]
    return(rounded[0])

def Buy_Dessision_Multi_In_Out(input):
    A=np.array(input)
    predictions = model.predict(normalize(A))
    rounded = [round(x[0]) for x in predictions]
    return(rounded)

def Buy_Dessision_Multi_In_Out_Normalized(input):
    A=np.array(input)
    predictions = model.predict(A)
    rounded = [round(x[0]) for x in predictions]
    return(rounded)

## Gloabale variables

In [9]:
Normalization=None
Model_FileName='XcryptoAi_model.hdf5'
Normalization_File='Normalization_Values.json'
Binance_USDT_HALAL = [
    "BTC/USDT",
    "LUNA/USDT",
    "ETH/USDT",
    "GMT/USDT",
    "UST/USDT",
    "SOL/USDT",
    "APE/USDT",
    "XRP/USDT",
    "IDEX/USDT",
    "AVAX/USDT",
    "DOT/USDT",
    "ADA/USDT",
    "JASMY/USDT",
    "TRX/USDT",
    "NEAR/USDT",
    "AXS/USDT",
    "GAL/USDT",
    "GALA/USDT",
    "SHIB/USDT",
    "ZIL/USDT",
    "ENS/USDT",
    "DOGE/USDT",
    "LTC/USDT",
    "EUR/USDT",
    "MANA/USDT",
    "DAR/USDT",
    "WAVES/USDT",
    "LAZIO/USDT",
    "ALICE/USDT",
    "ROSE/USDT",
    "ZEC/USDT",
    "ALGO/USDT",
    "GRT/USDT",
    "PSG/USDT",
    "SLP/USDT",
    "EOS/USDT",
    "PORTO/USDT",
    "ICP/USDT",
    "EGLD/USDT",
    "XMR/USDT",
    "KDA/USDT",
    "ETC/USDT",
    "MBOX/USDT",
    "OGN/USDT",
    "AR/USDT",
    "GLMR/USDT",
    "LOKA/USDT",
    "XLM/USDT",
    "MTL/USDT",
    "SNX/USDT",
    "PYR/USDT",
    "DASH/USDT",
    "CITY/USDT",
    "ASTR/USDT",
    "IOTA/USDT",
    "ACM/USDT",
    "BAR/USDT",
    "JUV/USDT",
    "SYS/USDT",
    "RVN/USDT",
    "MBL/USDT",
    "REN/USDT",
    "JST/USDT",
    "OMG/USDT",
    "ATM/USDT",
    "XEC/USDT",
    "STORJ/USDT",
    "ZRX/USDT",
    "SRM/USDT",
    "ICX/USDT",
    "API3/USDT",
    "ONT/USDT",
    "SKL/USDT",
    "MULTI/USDT",
    "QTUM/USDT",
    "COCOS/USDT",
    "VOXEL/USDT",
    "HIVE/USDT",
    "KP3R/USDT",
    "ATA/USDT",
    "STMX/USDT",
    "ADX/USDT",
    "HIGH/USDT",
    "NULS/USDT",
    "MLN/USDT",
    "YGG/USDT",
    "SC/USDT",
    "CKB/USDT",
    "TOMO/USDT",
    "STX/USDT",
    "FLUX/USDT",
    "DNT/USDT",
    "ORN/USDT",
    "PLA/USDT",
    "BADGER/USDT",
    "DF/USDT",
    "MOB/USDT",
    "LPT/USDT",
    "SCRT/USDT",
    "RAD/USDT",
    "NMR/USDT",
    "ELF/USDT",
    "TORN/USDT",
    "T/USDT",
    "QUICK/USDT",
    "LSK/USDT",
    "FIDA/USDT",
    "XNO/USDT",
    "BTG/USDT",
    "GHST/USDT",
    "EPS/USDT"
]

pair_list = Binance_USDT_HALAL


In [10]:
try:
    MetaData=pd.read_csv("D:\+DATA+\MetaData.csv")
except:
    MetaData=get_crypto_metadata(Binance_USDT_HALAL)


## loding the model

In [11]:
model = load_model(Model_FileName)

Loding Data

In [12]:
df = pd.read_csv('A:/UltimeTradingBot/Crypto_backtest_tools/cleaned_w15_50percent_novolume.csv',index_col=0)
# high_weight=3
# df["high"]=(df["open"]+high_weight*df["high"]+df["low"]+df["close"])/(3+high_weight)
# df.rename(columns={"high":"price"},inplace = True)
# df["BTC_high"]=(df["BTC_open"]+high_weight*df["BTC_high"]+df["BTC_low"]+df["BTC_close"])/(3+high_weight)
# df.rename(columns={"BTC_high":"BTC_price"},inplace = True)
# df=df.drop(columns=["BTC_open","BTC_low","BTC_close","open","low","close"])


KeyboardInterrupt: 

In [ ]:
df.iloc[0]

price          0.258300
high-1        -0.006194
low-1          0.001549
open-1        -0.002710
close-1       -0.004259
                ...    
day            1.000000
hour          15.000000
minute        18.000000
lunch_day   -117.000000
buy            1.000000
Name: 5, Length: 747, dtype: float64

In [ ]:
# for key in df.keys():
#     if key.find("BTC")!=-1 and (key.find("open")!=-1 or
#     key.find("high")!=-1 or key.find("low")!=-1 or key.find("close")!=-1):
#         df[key]=(df["BTC_price"]-df[key])/df["BTC_price"]
#     if key.find("BTC")==-1 and (key.find("open")!=-1 or
#     key.find("high")!=-1 or key.find("low")!=-1 or key.find("close")!=-1):
#         df[key]=(df["price"]-df[key])/df["price"]

In [ ]:
#df['volume'][0]
# VolRemover=["volume","volume-1","BTC_volume","BTC_volume-1"]
# #VolRemover=[]
# for key in df.keys():
#     if key.find("volume-1_") != -1 :
#         VolRemover.append(key)
# print(VolRemover)
# df=df.drop(columns=VolRemover)# 

In [ ]:
df[df['day']==3][df["buy"]==1]

C:\Users\ayoub\AppData\Local\Temp\ipykernel_4636\1716042835.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[df['day']==3][df["buy"]==1]


,price,high-1,low-1,open-1,close-1,high-2,low-2,open-2,close-2,volume-2,...,BTC_high-15_5min,BTC_low-15_5min,BTC_open-15_5min,BTC_close-15_5min,BTC_volume-15_5min,day,hour,minute,lunch_day,buy
25,0.266187,-0.006700,-0.004446,-0.005723,-0.004483,-0.007263,-0.004784,-0.006061,-0.005723,8.135990e+04,...,-0.004625,-0.002497,-0.004625,-0.002908,121.085350,3,7,7,-617,1
51,0.000093,-0.003151,0.000609,-0.003151,0.000609,-0.003151,0.000609,0.000501,-0.002291,4.693775e+08,...,-0.005787,-0.003975,-0.004048,-0.005608,70.892530,3,0,34,-611,1
68,55.441667,-0.001233,0.001653,-0.001233,0.000752,-0.000691,0.001653,0.001653,-0.000691,2.454000e+01,...,0.001966,0.002985,0.002007,0.002834,69.386300,3,10,5,-499,1
100,8.983333,-0.002523,-0.000074,-0.002189,-0.000186,-0.002635,0.000037,-0.000631,-0.002635,5.411000e+02,...,0.009829,0.012100,0.010704,0.010801,289.076130,3,15,50,-420,1
174,21.766667,-0.000153,-0.000153,-0.000153,-0.000153,-0.000153,-0.000153,-0.000153,-0.000153,0.000000e+00,...,-0.014767,-0.012131,-0.014242,-0.013426,165.612740,3,16,39,-527,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1084258,0.727883,-0.001809,0.003686,-0.000435,0.001626,-0.004694,-0.000435,-0.004694,-0.000435,2.175641e+04,...,-0.014319,-0.002560,-0.003060,-0.014101,901.091280,3,7,16,527,1
1084280,0.033890,-0.002656,-0.000590,-0.002066,-0.001180,-0.007377,-0.001475,-0.006787,-0.002066,3.864282e+06,...,-0.004209,0.000003,-0.003359,-0.001068,273.401745,3,2,21,569,1
1084283,0.048237,-0.003801,0.001175,-0.002764,0.001175,-0.006288,-0.002971,-0.006288,-0.002971,1.847130e+04,...,-0.008375,-0.000885,-0.006085,-0.008278,545.043657,3,14,2,-323,1
1084343,0.020591,-0.008021,0.002323,-0.005399,0.001643,-0.012489,-0.002388,-0.012489,-0.005982,1.260659e+06,...,-0.022357,-0.009742,-0.011860,-0.019319,806.548067,3,12,11,-187,1


Import Normalization values

In [ ]:
for k in df.keys():print(k,end='        ')

price        high-1        low-1        open-1        close-1        high-2        low-2        open-2        close-2        volume-2        high-3        low-3        open-3        close-3        volume-3        high-4        low-4        open-4        close-4        volume-4        high-5        low-5        open-5        close-5        volume-5        high-6        low-6        open-6        close-6        volume-6        high-7        low-7        open-7        close-7        volume-7        high-8        low-8        open-8        close-8        volume-8        high-9        low-9        open-9        close-9        volume-9        high-10        low-10        open-10        close-10        volume-10        high-11        low-11        open-11        close-11        volume-11        high-12        low-12        open-12        close-12        volume-12        high-13        low-13        open-13        close-13        volume-13        high-14        low-14        open-14        clo

In [ ]:
dt=df.drop(columns=["buy"]).to_numpy()

In [ ]:
dt.shape

(429940, 746)

In [14]:
Y=df["buy"].to_numpy()

NameError: name 'df' is not defined

In [13]:
print(dt.shape)

NameError: name 'dt' is not defined

In [ ]:
print(Y.shape)

(429940,)


## Try once

In [ ]:

index_sample=5123
#print(df.iloc[index_sample])
Sample=df.drop(columns=["buy"]).iloc[index_sample]
with tf.device('/gpu:0'):
    Bgpu=Buy_Dessision(Sample)
# with tf.device('/cpu:0'):
#     Bcpu=Buy_Dessision(Sample)
# Bcpu
verdict=Bgpu==Y[index_sample]
print("verdict ---> "+str(verdict))


normalization is loaded
verdict ---> True


In [ ]:
pdata.iloc[0].shape

(746,)

## no GPU

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
if(df.iloc[33,0]==dt[33,0]):dt=normalize(dt)

normalization is loaded


In [ ]:
with tf.device('/cpu:0'):
    accuracy = model.evaluate(dt, Y)
print("accuracy: "+str(accuracy[1]))


13436/13436 [==============================] - 113s 8ms/step - loss: 0.0471 - accuracy: 0.9841
accuracy: 0.9841163754463196


In [ ]:
Y

In [ ]:
with tf.device('/gpu:0'):
    accuracy = model.evaluate(dt[110:int(len(dt)/9),:], Y[110:int(len(dt)/9)])
print("accuracy: "+str(accuracy[1]))


1490/1490 [==============================] - 7s 5ms/step - loss: 0.0397 - accuracy: 0.9917
accuracy: 0.9917122721672058


In [ ]:
Exchange=ccxt.binance()
ex=Exchange

In [ ]:
pair='GMT/BUSD'
# bitcoin_ticker = Exchange.fetch_ticker('BTC/USDT')
# btc_current_price=bitcoin_ticker['info']['askPrice']
# print(btc_current_price)




# li1 = df.keys().values.tolist()
# li2 = fulldata.keys().values.tolist()
 
# s = set(li2)
# temp3 = [x for x in li1 if x not in s]
# print(temp3)

18462.66000000


In [ ]:
Timestamp=pd.to_datetime(ex.fetchTime(),unit='ms')
Timestamp.dayofweek+1
Timestamp.hour
Timestamp.minute

19

In [ ]:
pd.to_datetime("25/09/2022").dayofweek+1

7

In [ ]:
pd.to_datetime("26/09/2022").dayofweek+1

1

In [ ]:
def instant_full_data(pair,exchange=ex,window=15):
    pdata=instant_pair_data(pair,exchange=exchange,window=15)
    btcdata=instant_pair_data("BTC/USDT",exchange=exchange,window=15).add_prefix("BTC_")
    Timestamp=pd.to_datetime(ex.fetchTime(),unit='ms')
    pdata=pd.concat([pdata,btcdata],axis=1)
    pdata.loc[0,"day"]=Timestamp.dayofweek+1
    pdata.loc[0,"hour"]=Timestamp.hour
    pdata.loc[0,"minute"]=Timestamp.minute
    pdata.loc[0,"lunch_day"]=int(-(pd.to_datetime(MetaData[MetaData["Pair"] == (pair.split("/")[0]+"/USDT")]["launch_minute"])-pd.Timestamp('2020-01-01 00:00:00.000000')).dt.days)
    for key in pdata.keys():
        if key.find("BTC")!=-1 and (key.find("open")!=-1 or
        key.find("high")!=-1 or key.find("low")!=-1 or key.find("close")!=-1):
            pdata[key]=(pdata["BTC_price"]-pdata[key])/pdata["BTC_price"]
        if key.find("BTC")==-1 and (key.find("open")!=-1 or
        key.find("high")!=-1 or key.find("low")!=-1 or key.find("close")!=-1):
            pdata[key]=(pdata["price"]-pdata[key])/pdata["price"]
    return pdata

In [ ]:
import time
import warnings
warnings.filterwarnings('ignore')
PRERR=False
def prerr(err):
    if PRERR:
        print("\033[0;31m Error in "+str(sys._getframe().f_code.co_name) +" \033[0;33m"+str(err))

GoodDeal={}

while True:
    pdata=instant_full_data(pair,exchange=ex,window=15)
    if Buy_Dessision(pdata.iloc[0]) == 1:
        print("buying")
        pp=0
        for i in range(14):
            time.sleep(30)
            pp=ex.fetch_ticker(pair).ticker['info']['askPrice']
            if(float(pdata.loc[0,"price"]) >= float(pp)):
                print("+++ wining bought at:"+str(pdata.loc[0,"price"]) +" sold at: "+str(pp) )
                break
        print("--- losing bought at: "+str(pdata.loc[0,"price"]) +" sold at: "+str(pp) )
            
    time.sleep(30)

KeyboardInterrupt: 